In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from scipy.interpolate import lagrange
from PyEMD import EMD
from entropy import fuzzy_entropy

# Set Plotting Styles
plt.style.use('ggplot')

# Read the data from CSV file
newData = pd.read_csv("Office_Garman.csv", index_col="timestamp", parse_dates=True)
# Remove duplicate entries
newData = newData.drop_duplicates()
# Set the frequency to hourly
newData = newData.asfreq('H')
# Fill missing values using forward fill method
newData = newData.fillna(method='ffill')

# Visualizing Energy Consumption Data
plt.plot(newData.index, newData['Office_Garman'])
plt.title('Energy Consumption')
plt.ylabel('Energy Consumption (MW)')
plt.show()

# Define the split date
split_date = '2015-09-30'

# Split the data into training and testing sets
train = newData.loc[:split_date]
test = newData.loc[split_date:]

# DBSCAN detection
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(train['Office_Garman'].values.reshape(-1, 1))
labels = dbscan.labels_

# Lagrange interpolation
train_interpolated = train.copy()
train_interpolated['Office_Garman'] = lagrange(train_interpolated['Office_Garman'], train_interpolated.index)

# EMD decomposition
emd = EMD()
components = emd(train_interpolated['Office_Garman'].values)

# FuzzyEn entropy calculation
entropy_values = []
for component in components:
    entropy = fuzzy_entropy(component)
    entropy_values.append(entropy)

# Divide components into high-frequency and low-frequency
high_frequency = components[0]  # Modify this based on the specific component
low_frequency = components[1]  # Modify this based on the specific component

# Calculate actual energy consumption value
actual_energy_consumption = high_frequency + low_frequency

# Progress bar
def progress_bar(current, total):
    percent = (current / total) * 100
    print(f"Progress: {current}/{total} ({percent:.2f}%)")

# Example usage of progress bar
total_tasks = 10
for i in range(total_tasks):
    # Perform a task
    progress_bar(i + 1, total_tasks)
    # Do other computations

# Update the code above with the specific modifications required for your dataset and algorithms.


ModuleNotFoundError: No module named 'PyEMD'